https://gitee.com/pondali/LeetCode-Py

# Brute Force 算法

暴力匹配算法，也可以叫做朴素匹配算法。

算法思想：对于给定文本串 T 与模式串 p，从文本串的第一个字符开始与模式串 p 的第一个字符进行比较，如果相等，则继续逐个比较后续字符，否则从文本串 T 的第二个字符起重新和模式串 p 进行比较。依次类推，直到模式串 p 中每个字符依次与文本串 T 的一个连续子串相等，则模式匹配成功。否则模式匹配失败。

时间复杂度是 O(n∗m)

In [1]:
def bruteForce(T: str, p: str) -> int:
    n, m = len(T), len(p)
    
    i, j = 0, 0                     # i 表示文本串 T 的当前位置，j 表示模式串 p 的当前位置
    while i < n and j < m:          # i 或 j 其中一个到达尾部时停止搜索
        if T[i] == p[j]:            # 如果相等，则继续进行下一个字符匹配
            i += 1
            j += 1
        else:
            i = i - (j - 1)         # 如果匹配失败则将 i 移动到上次匹配开始位置的下一个位置
            j = 0                   # 匹配失败 j 回退到模式串开始位置

    if j == m:
        return i - j                # 匹配成功，返回匹配的开始位置
    else:
        return -1                   # 匹配失败，返回 -1

# KMP 算法

 算法思想：对于给定文本串 T 与模式串 p，当发现文本串 T 的某个字符与模式串 p 不匹配的时候，可以利用匹配失败后的信息，尽量减少模式串与文本串的匹配次数，避免文本串位置的回退，以达到快速匹配的目的。

 时间复杂度是 O(n+m)

In [4]:
def get_next(sub):
    l = [-1]
    for i in range(1,len(sub)):
        j = i - 1
        while j > 0:
            if sub[:j] == sub[:i][-j:]:
                break
            j -= 1
        l.append(j)
    return l


def kmp(main, sub):
    n = get_next(sub)
    i = j =  0
    while i < len(main) and j < len(sub):
        if main[i] == sub[j]:
            i, j = i+1, j+1
        else:
            j = n[j]
            if j == -1:
                i, j = i + 1, j + 1
    return i-j+1 if j >= len(sub) else 0


sub = "ababc"
main = "abaaababcee"
loc = kmp(main,sub)
print(main)
print(sub.rjust(loc+len(sub)-1,))
print("第{0}位匹配".format(loc))

abaaababcee
    ababc
第5位匹配


In [ ]:
#!/usr/bin/env python3
# -*- encoding: utf-8 -*-

# 字符串匹配-KMP：指定关键列数据(String)，识别相互匹配的重复项并删除

# 导入相应数据库客户端
import mysql.connector as mdb
import pandas as pd
from fuzzywuzzy import process

# 设置目标 table 查询语句以提取数据
targettable = "name"

# 使用 mysql api 和 read_sql() 从 mysql 获取和加载数据
host = "pdccmysql"
user = "pdccuser"
password = "pdccpass"
database = "pdccschema"
connmysql = mdb.connect(host=host,database=database,user=user,password=password)
df = pd.read_sql_table(table_name=targettable,con=connmysql)    # 生成 DataFrame
connmysql.close()

# 将 DataFrame 中的一个或多个列设置为索引
df.set_index(["column1_name"], inplace=True)

# KMP 算法
def get_next(sub):
    l = [-1]
    for i in range(1,len(sub)):
        j = i - 1
        while j > 0:
            if sub[:j] == sub[:i][-j:]:
                break
            j -= 1
        l.append(j)
    return l

def kmp(main, sub):
    n = get_next(sub)
    i = j =  0
    while i < len(main) and j < len(sub):
        if main[i] == sub[j]:
            i, j = i+1, j+1
        else:
            j = n[j]
            if j == -1:
                i, j = i + 1, j + 1
    return i-j+1 if j >= len(sub) else 0

# 查找重复字符串函数
def find_duplicates(dataframe, column_name):
    """
    检测DataFrame中指定列的重复项。
    :param dataframe: 要处理的DataFrame。
    :param column_name: 要检测重复项的列名。
    :return: 包含重复数据的索引列表。
    """
    duplicates = set()
    for i in range(len(dataframe)):
        for j in range(i + 1, len(dataframe)):
            # 计算当前行与所有行字符串是否匹配
            match = kmp(dataframe[column_name].iloc[i], dataframe[column_name])
            # 若匹配则认为是重复数据
            if match[1] != -1:
                duplicates.add(j)
    return list(duplicates)

# 调用函数并打印结果
duplicates_indices = find_duplicates(df, 'column_str')
df_cleaned = df.drop(duplicates_indices).reset_index(drop=True)

# 写入数据库，method = 'multi' 需要数据库支持 
df.to_sql(name=targettable,con=connmysql,if_exists='replace',chunksize=1000)
connmysql.close()

In [10]:
def kmp(m_str, s_str):
    # m_str表示主串，s_str表示模式串

    # 求next数组
    next_ls = [-1]*len(s_str)
    m = 1  # 从1开始匹配
    s = 0
    next_ls[1]=0
    while m<len(s_str)-1:
        if s_str[m] == s_str[s] or s == -1:
            m += 1
            s += 1
            next_ls[m] = s
        else:
            s = next_ls[s]
    #  print(next_ls)  检查next数组
    # KMP
    i = j = 0  # i,j位置指针初始值为0
    while i < len(m_str) and j < len(s_str):
        # 模式串遍历结束匹配成功，主串遍历结束匹配失败
        # 匹配成功或失败后退出
        if m_str[i] == s_str[j] or j == -1:
            # 把j==-1时纳入到条件判断中，实现i+1，j归零
            i += 1
            j += 1
        else:
            j = next_ls[j]

    if j == len(s_str):
        return i - j  # 匹配成功
    return -1  # 匹配失败

#测试
print(kmp('decdagee','age'))

4


In [9]:
# 生成 next 数组
# next[j] 表示下标 j 之前的模式串 p 中，最长相等前后缀的长度
def generateNext(p: str):
    m = len(p)
    next = [0 for _ in range(m)]                # 初始化数组元素全部为 0
    
    left = 0                                    # left 表示前缀串开始所在的下标位置
    for right in range(1, m):                   # right 表示后缀串开始所在的下标位置
        while left > 0 and p[left] != p[right]: # 匹配不成功, left 进行回退, left == 0 时停止回退
            left = next[left - 1]               # left 进行回退操作
        if p[left] == p[right]:                 # 匹配成功，找到相同的前后缀，先让 left += 1，此时 left 为前缀长度
            left += 1
        next[right] = left                      # 记录前缀长度，更新 next[right], 结束本次循环, right += 1

    return next

# KMP 匹配算法，T 为文本串，p 为模式串
def kmp(T: str, p: str) -> int:
    n, m = len(T), len(p)
    
    next = generateNext(p)                      # 生成 next 数组
    
    j = 0                                       # j 为模式串中当前匹配的位置
    for i in range(n):                          # i 为文本串中当前匹配的位置
        while j > 0 and T[i] != p[j]:           # 如果模式串前缀匹配不成功, 将模式串进行回退, j == 0 时停止回退
            j = next[j - 1]
        if T[i] == p[j]:                        # 当前模式串前缀匹配成功，令 j += 1，继续匹配
            j += 1
        if j == m:                              # 当前模式串完全匹配成功，返回匹配开始位置
            return i - j + 1
    return -1                                   # 匹配失败，返回 -1
            
sub = "ababc"
main = "abaaababcee"
loc = kmp(main,sub)
print(main)
print(sub.rjust(loc+len(sub)))
print("第{0}位匹配".format(loc))

abaaababcee
    ababc
第4位匹配


# Rabin Karp 算法

算法思想：对于给定文本串 T 与模式串 p，通过滚动哈希算快速筛选出与模式串 p 不匹配的文本位置，然后在其余位置继续检查匹配项。

RK 算法可以看做是 BF 算法的一种改进。在 BF 算法中，每一个字符都需要进行比较。而在 RK 算法中，判断模式串的哈希值与每个子串的哈希值之间是否相等的时间复杂度为 O(1)。总共需要比较 n - m + 1 个子串的哈希值，所以 RK 算法的整体时间复杂度为 O(n)。跟 BF 算法相比，RK 算法的效率提高了很多。

但是如果存在冲突的情况下，算法的效率会降低。最坏情况是每一次比较模式串的哈希值和子串的哈希值时都相等，但是每一次都会出现冲突，那么每一次都需要验证模式串和子串每个字符是否完全相同，那么总的比较次数就是 m * (n - m + 1) ，时间复杂度就会退化为 O(m∗n)。

In [ ]:
# T 为文本串，p 为模式串，d 为字符集的字符种类数，q 为质数
def rabinKarp(T: str, p: str, d, q) -> int:
    n, m = len(T), len(p)
    if n < m:
        return -1
    
    hash_p, hash_t = 0, 0
    
    for i in range(m):
        hash_p = (hash_p * d + ord(p[i])) % q           # 计算模式串 p 的哈希值
        hash_t = (hash_t * d + ord(T[i])) % q           # 计算文本串 T 中第一个子串的哈希值
    
    power = pow(d, m - 1) % q                           # power 用于移除字符哈希时
    
    for i in range(n - m + 1):
        if hash_p == hash_t:                            # 检查模式串 p 的哈希值和子串的哈希值
            match = True                                # 如果哈希值相等，验证模式串和子串每个字符是否完全相同（避免哈希冲突）
            for j in range(m):
                if T[i + j] != p[j]:
                    match = False                       # 模式串和子串某个字符不相等，验证失败，跳出循环
                    break
            if match:                                   # 如果模式串和子串每个字符是否完全相同，返回匹配开始位置
                return i
        if i < n - m:                                   # 计算下一个相邻子串的哈希值
            hash_t = (hash_t - power * ord(T[i])) % q   # 移除字符 T[i]
            hash_t = (hash_t * d + ord(T[i + m])) % q   # 增加字符 T[i + m]
            hash_t = (hash_t + q) % q                   # 确保 hash_t >= 0
        
    return -1

# Boyer Moore 算法

算法思想：对于给定文本串 T 与模式串 p，先对模式串 p 进行预处理。然后在匹配的过程中，当发现文本串 T 的某个字符与模式串 p 不匹配的时候，根据启发策略，能够直接尽可能地跳过一些无法匹配的情况，将模式串多向后滑动几位。

- BM 算法在预处理阶段的时间复杂度为 O(n+σ)，其中 σ 是字符集的大小。
- BM 算法在搜索阶段最好情况是每次匹配时，模式串 p 中不存在与文本串 T 中第一个匹配的字符。这时的时间复杂度为 O(n/m)。
- BM 算法在搜索阶段最差情况是文本串 T 中有多个重复的字符，并且模式串 p 中有 m - 1 个相同字符前加一个不同的字符组成。这时的时间复杂度为 O(m∗n)。
- 当模式串 p 是非周期性的，在最坏情况下，BM 算法最多需要进行 3∗n 次字符比较操作。

In [12]:
# BM 匹配算法
def boyerMoore(T: str, p: str) -> int:
    n, m = len(T), len(p)
    
    bc_table = generateBadCharTable(p)              # 生成坏字符位置表
    gs_list = generageGoodSuffixList(p)             # 生成好后缀规则后移位数表
    
    i = 0
    while i <= n - m:
        j = m - 1
        while j > -1 and T[i + j] == p[j]:          # 进行后缀匹配，跳出循环说明出现坏字符
            j -= 1
        if j < 0:
            return i                                # 匹配完成，返回模式串 p 在文本串 T 中的位置
        bad_move = j - bc_table.get(T[i + j], -1)   # 坏字符规则下的后移位数
        good_move = gs_list[j]                      # 好后缀规则下的后移位数
        i += max(bad_move, good_move)               # 取两种规则下后移位数的最大值进行移动
    return -1
            
    
# 生成坏字符位置表
# bc_table[bad_char] 表示坏字符在模式串中最后一次出现的位置
def generateBadCharTable(p: str):
    bc_table = dict()
    
    for i in range(len(p)):
        bc_table[p[i]] = i                          # 更新坏字符在模式串中最后一次出现的位置
    return bc_table

# 生成好后缀规则后移位数表
# gs_list[j] 表示在 j 下标处遇到坏字符时，可根据好规则向右移动的距离
def generageGoodSuffixList(p: str):
    # 好后缀规则后移位数表
    # 情况 1: 模式串中有子串匹配上好后缀
    # 情况 2: 模式串中无子串匹配上好后缀，但有最长前缀匹配好后缀的后缀
    # 情况 3: 模式串中无子串匹配上好后缀，也找不到前缀匹配
    
    m = len(p)
    gs_list = [m for _ in range(m)]                 # 情况 3：初始化时假设全部为情况 3
    suffix = generageSuffixArray(p)                 # 生成 suffix 数组
    
    j = 0                                           # j 为好后缀前的坏字符位置
    for i in range(m - 1, -1, -1):                  # 情况 2：从最长的前缀开始检索
        if suffix[i] == i + 1:                      # 匹配到前缀，即 p[0...i] == p[m-1-i...m-1]
            while j < m - 1 - i:
                if gs_list[j] == m:
                    gs_list[j] = m - 1 - i          # 更新在 j 处遇到坏字符可向后移动位数
                j += 1
        
    for i in range(m - 1):                          # 情况 1：匹配到子串 p[i-s...i] == p[m-1-s, m-1]
        gs_list[m - 1 - suffix[i]] = m - 1 - i      # 更新在好后缀的左端点处遇到坏字符可向后移动位数
    return gs_list

# 生成 suffix 数组
# suffix[i] 表示为以下标 i 为结尾的子串与模式串后缀匹配的最大长度
def generageSuffixArray(p: str):
    m = len(p)
    suffix = [m for _ in range(m)]                  # 初始化时假设匹配的最大长度为 m
    for i in range(m - 2, -1, -1):                  # 子串末尾从 m - 2 开始
        start = i                                   # start 为子串开始位置
        while start >= 0 and p[start] == p[m - 1 - i + start]:
            start -= 1                              # 进行后缀匹配，start 为匹配到的子串开始位置
        suffix[i] = i - start                       # 更新以下标 i 为结尾的子串与模式串后缀匹配的最大长度
    return suffix

print(boyerMoore("abbcfdddbddcaddebc", "dddb"))
print(boyerMoore("", ""))

5
0


# Horspool 算法

算法思想：对于给定文本串 T 与模式串 p，先对模式串 p 进行预处理。然后在匹配的过程中，当发现文本串 T 的某个字符与模式串 p 不匹配的时候，根据启发策略，能够尽可能的跳过一些无法匹配的情况，将模式串多向后滑动几位。

Horspool 算法在平均情况下的时间复杂度为 O(n)，但是在最坏情况下时间复杂度会退化为 O(n∗m)。

In [13]:
# horspool 算法，T 为文本串，p 为模式串
def horspool(T: str, p: str) -> int:
    n, m = len(T), len(p)
    
    bc_table = generateBadCharTable(p)          # 生成后移位数表
    
    i = 0
    while i <= n - m:
        j = m - 1
        while j > -1 and T[i + j] == p[j]:      # 进行后缀匹配，跳出循环说明出现坏字符
            j -= 1
        if j < 0:
            return i                            # 匹配完成，返回模式串 p 在文本串 T 中的位置
        i += bc_table.get(T[i + m - 1], m)      # 通过后移位数表，向右进行进行快速移动
    return -1                                   # 匹配失败

# 生成后移位数表
# bc_table[bad_char] 表示遇到坏字符可以向右移动的距离
def generateBadCharTable(p: str):
    m = len(p)
    bc_table = dict()
    
    for i in range(m - 1):                      # 迭代到 m - 2
        bc_table[p[i]] = m - 1 - i              # 更新遇到坏字符可向右移动的距离
    return bc_table

print(horspool("abbcfdddbddcaddebc", "aaaaa"))
print(horspool("abbcfdddbddcaddebc", "bcf"))

-1
2


# Sunday 算法

算法思想：对于给定文本串 T 与模式串 p，先对模式串 p 进行预处理。然后在匹配的过程中，当发现文本串 T 的某个字符与模式串 p 不匹配的时候，根据启发策略，能够尽可能的跳过一些无法匹配的情况，将模式串多向后滑动几位。

平均情况下的时间复杂度为 O(n)，但是在最坏情况下时间复杂度会退化为 O(n∗m)。

In [14]:
# sunday 算法，T 为文本串，p 为模式串
def sunday(T: str, p: str) -> int:
    n, m = len(T), len(p)
    
    bc_table = generateBadCharTable(p)          # 生成后移位数表
    
    i = 0
    while i <= n - m:
        j = 0
        if T[i: i + m] == p:
            return i                            # 匹配完成，返回模式串 p 在文本串 T 的位置
        if i + m >= n:
            return -1
        i += bc_table.get(T[i + m], m + 1)      # 通过后移位数表，向右进行进行快速移动
    return -1                                   # 匹配失败

# 生成后移位数表
# bc_table[bad_char] 表示遇到坏字符可以向右移动的距离
def generateBadCharTable(p: str):
    m = len(p)
    bc_table = dict()
    
    for i in range(m):                      # 迭代到最后一个位置 m - 1
        bc_table[p[i]] = m - i              # 更新遇到坏字符可向右移动的距离
    return bc_table

print(sunday("abbcfdddbddcaddebc", "aaaaa"))
print(sunday("abbcfdddbddcaddebc", "bcf"))

-1
2


# 总结

| 算法     | 效率            | 优点        | 缺点        | 适用环境                 |
|--------|---------------|-----------|-----------|----------------------|
| BF     | O(n*m)        | 简单实现      | 效率低       | 仅简单字符串匹配，不要求效率的情况    |
| RK     | O(n)~O(n*m)   | 串与串对比速度快  | 稳定性差、代码量大 | 模式串较长或文本串与模式串的差较小的情况 |
| KMP    | O(n+m)        | 效率高、稳定性强  | 无明显短板     | 要求稳定性或文本串较短的情况       |
| BM     | O(n/m)~O(n+m) | 效率极高      | 前期准备工作量大  | 文本串较长的情况             |
| Sunday | O(n/m)~O(n*m) | 代码简单且速度客观 | 稳定性差      | 模式串无大量重复的情况          |